In [1]:
from keras.datasets import imdb

Using TensorFlow backend.


In [4]:
import requests
requests.packages.urllib3.disable_warnings()
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
# Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
# Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

In [5]:
vocabulary_size = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)

17465344/17464789 [==============================] - 11s 1us/step


In [6]:
print(f'Loaded dataset with {len(X_train)} training samples, {len(X_test)} test samples.')

Loaded dataset with 25000 training samples, 25000 test samples.


In [7]:
print('---review---')
print(X_train[6])
print('---label---')
print(y_train[6])

---review---
[1, 2, 365, 1234, 5, 1156, 354, 11, 14, 2, 2, 7, 1016, 2, 2, 356, 44, 4, 1349, 500, 746, 5, 200, 4, 4132, 11, 2, 2, 1117, 1831, 2, 5, 4831, 26, 6, 2, 4183, 17, 369, 37, 215, 1345, 143, 2, 5, 1838, 8, 1974, 15, 36, 119, 257, 85, 52, 486, 9, 6, 2, 2, 63, 271, 6, 196, 96, 949, 4121, 4, 2, 7, 4, 2212, 2436, 819, 63, 47, 77, 2, 180, 6, 227, 11, 94, 2494, 2, 13, 423, 4, 168, 7, 4, 22, 5, 89, 665, 71, 270, 56, 5, 13, 197, 12, 161, 2, 99, 76, 23, 2, 7, 419, 665, 40, 91, 85, 108, 7, 4, 2084, 5, 4773, 81, 55, 52, 1901]
---label---
1


In [9]:
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print('---review with words--')
print([id2word.get(i, ' ') for i in X_train[6]])
print('--label--')
print(y_train[6])

1646592/1641221 [==============================] - 1s 1us/step
---review with words--
['the', 'and', 'full', 'involving', 'to', 'impressive', 'boring', 'this', 'as', 'and', 'and', 'br', 'villain', 'and', 'and', 'need', 'has', 'of', 'costumes', 'b', 'message', 'to', 'may', 'of', 'props', 'this', 'and', 'and', 'concept', 'issue', 'and', 'to', "god's", 'he', 'is', 'and', 'unfolds', 'movie', 'women', 'like', "isn't", 'surely', "i'm", 'and', 'to', 'toward', 'in', "here's", 'for', 'from', 'did', 'having', 'because', 'very', 'quality', 'it', 'is', 'and', 'and', 'really', 'book', 'is', 'both', 'too', 'worked', 'carl', 'of', 'and', 'br', 'of', 'reviewer', 'closer', 'figure', 'really', 'there', 'will', 'and', 'things', 'is', 'far', 'this', 'make', 'mistakes', 'and', 'was', "couldn't", 'of', 'few', 'br', 'of', 'you', 'to', "don't", 'female', 'than', 'place', 'she', 'to', 'was', 'between', 'that', 'nothing', 'and', 'movies', 'get', 'are', 'and', 'br', 'yes', 'female', 'just', 'its', 'because', 'ma

In [11]:
print(f'Maximum review length: {len(max((X_train + X_test), key=len))}')

Maximum review length: 2697


In [12]:
print(f'Minum review length: {len(min((X_test + X_test), key=len))}')

Minum review length: 14


In [13]:
from keras.preprocessing import sequence

In [14]:
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [16]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

In [17]:
embedding_size = 32
model = Sequential()
model.add(Embedding(vocabulary_size
                   , embedding_size
                   , input_length = max_words))
model.add(LSTM(100))
model.add(Dense(1
               , activation = 'sigmoid'))

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
model.compile(loss = 'binary_crossentropy'
             , optimizer = 'adam'
             , metrics = ['accuracy'])

In [19]:
batch_size = 64
num_epochs = 3

X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]

model.fit(X_train2
         , y_train2
         , validation_data = (X_valid, y_valid)
         , batch_size = batch_size
         , epochs = num_epochs)

/home/david/.local/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 24936 samples, validate on 64 samples
Epoch 1/3
24936/24936 [==============================] - 112s 4ms/step - loss: 0.4381 - accuracy: 0.7860 - val_loss: 0.1550 - val_accuracy: 0.9531
Epoch 2/3
24936/24936 [==============================] - 111s 4ms/step - loss: 0.2973 - accuracy: 0.8800 - val_loss: 0.2637 - val_accuracy: 0.9062
Epoch 3/3
24936/24936 [==============================] - 110s 4ms/step - loss: 0.2512 - accuracy: 0.9017 - val_loss: 0.1664 - val_accuracy: 0.9375


In [21]:
scores = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Accuracy: {scores[1]}')

25000/25000 [==============================] - 50s 2ms/step
Test Accuracy: 0.8755599856376648
